In [1]:
import tkinter as tk
from tkinter import messagebox
import lib_scen_intersection as lsi
import lib_map_draw as dr
import lib_spawn as spb
import carla
from importlib import reload
reload(lsi) #per refreshare modifiche import
reload(dr)
reload(spb)

class TrafficSimulatorGUI:
    def __init__(self, root, spacing=10):
        self.root = root
        self.root.title("Junction Scenarios")
        self.spacing = spacing
        self.world, self.crossings = self.connect_sim();
        

        # Array per gli incroci e gli spawn
        #self.crossings = ["Incrocio 1", "Incrocio 2", "Incrocio 3"]
        self.spawns = ["Spawn 1", "Spawn 2", "Spawn 3"]
        self.wp_pairs = None
        self.crossing_index = 0
        self.spawn_index = 0

        self.spawn_ego = None
        

        # Bottone per resettare il simulatore
        tk.Button(root, text="Resetta simulatore", command=self.reset_simulator).grid(row=0, column=0, columnspan=4, pady=self.spacing)

        # Incrocio - bottoni
        self.create_navigation_section("Incrocio", self.crossings,
                                       "Incrocio precedente", "Incrocio successivo", 1)

        # Spawn - bottoni
        self.create_navigation_section("Spawn", self.spawns,
                                       "Spawn precedente", "Spawn successivo", 2)

        # Bottone per "Spwan veicolo" e checkbox
        tk.Button(root, text="Spwan veicolo", command=self.spawn_vehicle).grid(row=3, column=0, pady=self.spacing)
        self.ignore_traffic_lights = tk.BooleanVar(value=False)  # Variabile per lo stato della checkbox
        tk.Checkbutton(root, text="Ignora semafori", variable=self.ignore_traffic_lights).grid(row=3, column=1, padx=self.spacing)

        # Textbox e bottone per "Genera traffico"
        tk.Label(root, text="Genera traffico:").grid(row=4, column=0, padx=self.spacing, pady=self.spacing)
        self.traffic_entry = tk.Entry(root, width=8)  # Larghezza della TextBox modificata
        self.traffic_entry.grid(row=4, column=1, padx=self.spacing, pady=self.spacing)
        tk.Button(root, text="Genera traffico", command=self.generate_traffic).grid(row=4, column=2, padx=self.spacing, pady=self.spacing)

        # Menù a tendina per profilo guida spwan
        tk.Label(root, text="Profilo guida spwan:").grid(row=5, column=0, padx=self.spacing, pady=self.spacing)
        self.spawn_profile = tk.StringVar(value="Seleziona")
        spawn_menu = tk.OptionMenu(root, self.spawn_profile, "Cautious", "Normal", "Aggressive",
                                   command=self.display_spawn_profile)
        spawn_menu.grid(row=5, column=1, padx=self.spacing, pady=self.spacing)

        # Menù a tendina per profilo guida traffico
        tk.Label(root, text="Profilo guida traffico:").grid(row=6, column=0, padx=self.spacing, pady=self.spacing)
        self.traffic_profile = tk.StringVar(value="Seleziona")
        traffic_menu = tk.OptionMenu(self.root, self.traffic_profile, "Cautious", "Normal", "Aggressive",
                                     command=self.display_traffic_profile)
        traffic_menu.grid(row=6, column=1, padx=self.spacing, pady=self.spacing)

        # Bottone per avviare la simulazione
        tk.Button(root, text="Avvia simulazione", command=self.start_simulation).grid(row=7, column=0, columnspan=4, pady=self.spacing)

    #ausiliaria per bottoni di scela di incroci e spawn
    def create_navigation_section(self, label, array, left_text, right_text, row):
        tk.Label(self.root, text=f"{label} attuale:").grid(row=row, column=0, padx=self.spacing, pady=self.spacing)
        #etichette all'avvio
        if label == "Incrocio":
           display_label = tk.Label(self.root, text=str(self.crossing_index  + 1 ))#text=array[0][0])
           # dato l'incrocio, recupero la lista di coppie di wp che lo circondano indicizzate da self.crossing_index
           self.wp_pairs = lsi.group_pairs_by_second_waypoint(self.crossings[self.crossing_index][1].get_waypoints(carla.LaneType.Driving) )
           self.spawns = spb.list_all_behind_wp_couple(self.world, self.wp_pairs, 10, 0)
           dr.focus_above(self.world, self.spawns[0], 60, -90)
            
        elif label == "Spawn":
            display_label = tk.Label(self.root, text=array[0])
            
        display_label.grid(row=row, column=1, padx=self.spacing, pady=self.spacing)

        def update_label(direction):
            nonlocal display_label
            if label == "Incrocio":
                self.crossing_index = (self.crossing_index + direction) % len(self.crossings)
                display_label.config(text=str(self.crossing_index  + 1 ) )#self.crossings[self.crossing_index][0]) #hunction contiene coppie id, coordinate
                # dato l'incrocio, recupero la lista di coppie di wp che lo circondano indicizzate da self.crossing_index
                self.wp_pairs = lsi.group_pairs_by_second_waypoint(self.crossings[self.crossing_index][1].get_waypoints(carla.LaneType.Driving) )
                self.spawns = spb.list_all_behind_wp_couple(self.world, self.wp_pairs, 10, 0)
                
                # Sposta la telecamera sull' primo wp della prima coppia dell'incrocio trovato
                dr.focus_above(self.world, self.spawns[0], 60, -90)
                
            elif label == "Spawn":
                self.spawn_index = (self.spawn_index + direction) % len(self.wp_pairs)
                display_label.config(text=str(self.spawn_index + 1))
                #spb.get_spawn_behind_wp(self.world, self.spawns[self.spawn_index][0], distance=10.0, focus=1)

        tk.Button(self.root, text=left_text, command=lambda: update_label(-1)).grid(row=row, column=2, padx=self.spacing, pady=self.spacing)
        tk.Button(self.root, text=right_text, command=lambda: update_label(1)).grid(row=row, column=3, padx=self.spacing, pady=self.spacing)

    def reset_simulator(self):
        # Resetta gli indici
        self.crossing_index = 0
        self.spawn_index = 0

        # Resetta i campi di input
        self.traffic_entry.delete(0, tk.END)

        # Resetta i profili selezionati
        self.spawn_profile.set("Seleziona")
        self.traffic_profile.set("Seleziona")

        # Resetta lo stato della checkbox
        self.ignore_traffic_lights.set(False)

        #cancella tutti gli attori dal simulatore
        lsi.reset_sim(self.world)

        # Messaggio di conferma
        messagebox.showinfo("Reset", "Il simulatore è stato resettato.")

    def spawn_vehicle(self):
        traffic_lights_status = "Semafori ignorati" if self.ignore_traffic_lights.get() else "Semafori osservati"
        messagebox.showinfo("Spwan Veicolo", f"Hello, world!\n{traffic_lights_status}")

    def generate_traffic(self):
        try:
            value = int(self.traffic_entry.get())
            if 10 <= value <= 50:
                messagebox.showinfo("Generazione Traffico", f"Generazione di {value} veicoli avviata!")
            else:
                raise ValueError
        except ValueError:
            messagebox.showerror("Errore", "Inserire un numero valido tra 10 e 50.")

    def display_spawn_profile(self, selection):
        messagebox.showinfo("Profilo Spwan", f"Profilo selezionato per Spwan: {selection}")

    def display_traffic_profile(self, selection):
        print("Profilo Traffico", f"Profilo selezionato per Traffico: {selection}")
        #messagebox.showinfo("Profilo Traffico", f"Profilo selezionato per Traffico: {selection} parametro {param}")

    def start_simulation(self):
        # Raccoglie i parametri selezionati
        crossing = self.crossings[self.crossing_index]
        spawn = self.spawns[self.spawn_index]
        traffic_value = self.traffic_entry.get()
        spawn_profile = self.spawn_profile.get()
        traffic_profile = self.traffic_profile.get()
        traffic_lights_status = "Semafori ignorati" if self.ignore_traffic_lights.get() else "Semafori osservati"

        # Controllo sull'input del traffico
        try:
            traffic_value = int(traffic_value)
            if not (10 <= traffic_value <= 50):
                raise ValueError
        except ValueError:
            traffic_value = "NON VALIDO (inserire un valore tra 10 e 50)"

        # Messaggio di riepilogo
        summary = (f"Simulazione avviata con i seguenti parametri:\n"
                   f"- Incrocio selezionato: {crossing}\n"
                   f"- Spawn selezionato: {spawn}\n"
                   f"- Veicoli da generare: {traffic_value}\n"
                   f"- Profilo guida spwan: {spawn_profile}\n"
                   f"- Profilo guida traffico: {traffic_profile}\n"
                   f"- Stato semafori: {traffic_lights_status}")
        messagebox.showinfo("Avvio Simulazione", summary)

    #connessione al simulatore 
    def connect_sim(self):
        client = carla.Client('localhost', 2000)
        client.set_timeout(5.0)
        world = client.get_world()
        bp_lib = world.get_blueprint_library() 
        map = world.get_map()
        
        # lista di wp alternati da dist di 2 metri
        waypoints = map.generate_waypoints(2.0)
        
        # array di incroci contenente coppie (id, oggetto_incrocio)
        unique_junctions = []
        
        # lettura dei wp eventualmente presenti ad un' intersezione
        #N.B.: più waypoint possono appartenere a uno stesso incrocio
        for waypoint in waypoints:
            if waypoint.is_junction:
                junction = waypoint.get_junction()
                if not(any(u[0] == junction.id for u in unique_junctions)):
                    unique_junctions.append( (junction.id, junction) )
        
        # Stampa il numero di incroci (9 distinti in town 10)
        print(f"Numero di incroci trovati: {len(unique_junctions)}\n") 
        
        return (world, unique_junctions)


# Avvio dell'interfaccia
root = tk.Tk()
app = TrafficSimulatorGUI(root, spacing=7)  # Personalizza la spaziatura qui
root.mainloop()


IndentationError: expected an indented block after 'for' statement on line 121 (lib_spawn.py, line 122)